In [ ]:
!pip install openai
!pip install python-dotenv
!pip install pandarallel
!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

from pandarallel import pandarallel
import os
import pandas as pd
import time
from tqdm.auto import tqdm
from dotenv import load_dotenv
import openai
from openai import OpenAI

In [ ]:
env_path = '/content/drive/MyDrive/toxic/secret.env'

load_dotenv(env_path)

api_key = os.getenv('MARITACA_API_KEY')

In [ ]:
client = openai.OpenAI(
    api_key=api_key,
    base_url="https://chat.maritaca.ai/api",
)

In [ ]:
prompt = """
              Você é um classificador de tweets ofensivos.
              Dado um tweet, responda apenas com 1 se for ofensivo ou com 0 se não for ofensivo.
              Sem explicações.
              Aqui estão 2 exemplos de tweets e suas classificações (entre parênteses):
              1. "minas gerais e o estado com maior numero de denuncias de crime e assedio eleitoral . a maioria esmagadora das denuncias tem conexão com a campanha de bolsonaro com a anuência do governador que também e acusado de fazer assedio a prefeitos e políticos mineiros ." (0)
              2. "não adianta derrubar ! todo mundo já sabe lula é ladrão . homicida corrupto mentiroso ateu terrorista chefe de falacias criminosas !" (1)
            """

In [ ]:
def classify_tweet(tweet_text):
  try:
    response = client.chat.completions.create(
      model = "sabia-3",
      messages = [
        {
          "role": "system",
          "content": prompt

        },
        {
          "role": "user",
          "content": tweet_text
        },
      ],
      max_tokens = 2,
      temperature = 0
      )
    return response.choices[0].message.content.strip()
  except Exception as e:
    return f"erro: {e}"

In [ ]:
# df = pd.read_excel("/content/drive/MyDrive/toxic/Base Bolsonaro Rotulada.xlsx")
df = pd.read_excel("/content/drive/MyDrive/toxic/Base Lula Rotulada.xlsx")

In [ ]:
df_resultado = df[["id", "text_original", "text_processed", "sabia-3"]]

In [ ]:
df = pd.read_excel('bolsonaro_sabia.xlsx')
df = df.drop(columns=['Unnamed: 0'])
df['id'] = df['id'].astype(str)
df.to_excel('bolsonaro_sabia.xlsx', index=False)

In [ ]:
client = openai.OpenAI(
    api_key=api_key,
    base_url="https://chat.maritaca.ai/api",
)

prompt = """
    Você é um classificador de tweets ofensivos. Dado um tweet, responda apenas com 1 se for ofensivo ou com 0 se não for ofensivo. Aqui estão 2 exemplos de tweets e suas classificações (entre parênteses):
    1. "Minas Gerais é o estado com maior número de denúncias de crime e assédio eleitoral. A maioria esmagadora das denúncias tem conexão com a campanha de Bolsonaro, com a anuência do governador, que também é acusado de fazer assédio a prefeitos e políticos mineiros." (0)
    2. "@user @user Não adianta derrubar! Todo mundo já sabe: Lula é ladrão. homicida, corrupto, mentiroso, ateu, terrorista, chefe de organizações criminosas!" (1)
    """

def classify_tweet(tweet_text, retry_count=3, wait_time=5):
    for attempt in range(retry_count):
        try:
            response = client.chat.completions.create(
                model = "sabia-3",
                messages = [
                    {
                        "role": "system",
                        "content": prompt
                    },
                    {
                        "role": "user",
                        "content": tweet_text
                    },
                ],
                max_tokens = 2,
                temperature = 0
            )
            return response
        except Exception as e:
            if attempt < retry_count - 1:
                time.sleep(wait_time)
            else:
                return "erro"

    return "erro"

In [ ]:
env_path = '/content/drive/MyDrive/toxic/secret.env'
load_dotenv(env_path)
api_key = os.getenv('MARITACA_API_KEY')
client = openai.OpenAI(
    api_key=api_key,
    base_url="https://chat.maritaca.ai/api",
)

prompt = """
Você é um classificador de tweets ofensivos. Dado um tweet, responda com os seguintes rótulos (0 para ausência, 1 para presença):
- insult: insulto, palavrões ou ofensas cujo objetivo é provocar, humilhar ou atacar alguém verbalmente.
- obscene: conteúdo vulgar, sexualmente explícito ou pornográfico.
- ideology: discurso de ódio baseado em ideologias.
- lgbtqphobia: discurso de ódio contra identidade de gênero ou orientação sexual.
- racism: discurso de ódio baseado em raça, etnia ou cor.
- sexism: discurso de ódio baseado em gênero ou sexo.
- xenophobia: discurso de ódio contra estrangeiros ou pessoas de outras culturas ou nacionalidades.
- other: ofensas não enquadradas acima.
Responda no formato:
insult:0,obscene:0,ideology:0,lgbtqphobia:0,racism:0,sexism:0,xenophobia:0,other:0
"""

In [ ]:


client = openai.OpenAI(api_key=api_key, base_url="https://chat.maritaca.ai/api")
ARQUIVO_LULA = "/content/drive/MyDrive/lula_2500k_anotmanual.xlsx"
ARQUIVO_BOLSONARO = "/content/drive/MyDrive/bolsonaro_2500k_anotmanual.xlsx"
COLUNA_ID = "id"
COLUNA_TWEET = "tweet"

prompt = """Você é um classificador de tweets ofensivos. Dado um tweet, responda com os seguintes rótulos (0 para ausência, 1 para presença):
- insult: insulto, palavrões ou ofensas cujo objetivo é provocar, humilhar ou atacar alguém verbalmente.
- obscene: conteúdo vulgar, sexualmente explícito ou pornográfico.
- ideology: discurso de ódio baseado em ideologias.
- lgbtqphobia: discurso de ódio contra identidade de gênero ou orientação sexual.
- racism: discurso de ódio baseado em raça, etnia ou cor.
- sexism: discurso de ódio baseado em gênero ou sexo.
- xenophobia: discurso de ódio contra estrangeiros ou pessoas de outras culturas ou nacionalidades.
- other: ofensas não enquadradas acima.

Responda no formato:
insult:0,obscene:0,ideology:0,lgbtqphobia:0,racism:0,sexism:0,xenophobia:0,other:0
"""

def classify_tweet(tweet_text, retry_count=3, wait_time=5):
    for attempt in range(retry_count):
        try:
            response = client.chat.completions.create(
                model="sabia-3",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": tweet_text}
                ],
                max_tokens=100,
                temperature=0
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if attempt < retry_count - 1:
                time.sleep(wait_time)
            else:
                return "erro"
    return "erro"

df_lula = pd.read_excel(ARQUIVO_LULA)
df_bolsonaro = pd.read_excel(ARQUIVO_BOLSONARO)

df = pd.concat([df_lula, df_bolsonaro], ignore_index=True)

df = df.dropna(subset=[COLUNA_ID, COLUNA_TWEET])

tqdm.pandas()
df["sabia"] = df[COLUNA_TWEET].progress_apply(classify_tweet)

df.to_excel("/content/drive/MyDrive/sabia_multilabel.xlsx", index=False)

In [ ]:
df["id"] = df["id"].astype(str)
df.to_excel("/content/drive/MyDrive/sabia_multilabel.xlsx", index=False)

In [ ]:
df_lula = pd.read_excel('/content/drive/MyDrive/lula_2500k_anotmanual.xlsx')
df_bolsonaro = pd.read_excel('/content/drive/MyDrive/bolsonaro_2500k_anotmanual.xlsx')